In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os
import warnings
import math
import tensorflow as tf
import gc
from tensorflow.keras.layers import Input, Dense, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanSquaredError
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Model
from sklearn.model_selection import KFold

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:f}'.format)
warnings.filterwarnings("ignore")
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# !unzip /content/drive/MyDrive/kaggle/ubiquant-market-prediction.zip -d /content

Archive:  /content/drive/MyDrive/kaggle/ubiquant-market-prediction.zip
  inflating: /content/example_sample_submission.csv  
  inflating: /content/example_test.csv  
  inflating: /content/train.csv      
  inflating: /content/ubiquant/__init__.py  
  inflating: /content/ubiquant/competition.cpython-37m-x86_64-linux-gnu.so  


In [2]:
chunk_size = math.pow(10, 6)
chunks_path = os.path.join(os.getcwd(), "drive/MyDrive/kaggle/chunks")

In [ ]:
def save_df_chunck(df: pd.DataFrame, index: int):
  data_columns = list(filter(lambda i: "f_" in i, df.columns.tolist()))
  print(f"Memory usage before df: {df.memory_usage().sum() / math.pow(1024, 2):.2f} MB")
  for i in data_columns:
    dtype = df[i].dtype
    if "float" in str(dtype):
      value_min = df[i].min()
      value_max = df[i].max()
      if value_min > np.finfo(np.float16).min and value_max < np.finfo(np.float16).max:
        df[i] = df[i].astype(np.float16)
      elif value_min > np.finfo(np.float32).min and value_max < np.finfo(np.float32).max:
        df[i] = df[i].astype(np.float32)
  print(f"Memory usage after df: {df.memory_usage().sum() / math.pow(1024, 2):.2f} MB")
  df.to_pickle(os.path.join(chunks_path, f"chunk_{index}.pkl"))

if not os.path.exists(chunks_path):
  os.mkdir(chunks_path)

for i, df in enumerate(pd.read_csv(os.path.join(os.getcwd(), "train.csv"), chunksize=chunk_size)):
  save_df_chunck(df, i)

In [3]:
!pip3 install pickle5
import pickle5 as pickle

     |████████████████████████████████| 256 kB 5.3 MB/s 


In [5]:
dfs = []

for i, _ in enumerate(os.listdir(chunks_path)):
  path = os.path.join(chunks_path, f"chunk_{i}.pkl")
  with open(path, "rb") as fh:
    data = pickle.load(fh)
    dfs.append(data)
    # dfs.append(pd.read_pickle(os.path.join(chunks_path, f"chunk_{i}.pkl")))
    
train_csv = pd.concat(dfs, axis=0, ignore_index=True)
print(f"train_csv memory usage: {np.round(train_csv.memory_usage().sum() / math.pow(1024,2), 2)} MB")
data_columns = list(filter(lambda i: "f_" in i, train_csv.columns.tolist()))
df = train_csv[:100000] # processing all rows requires too much system RAM so we'll use 100k rows
print(f"df memory usage: {np.round(df.memory_usage().sum() / math.pow(1024,2), 2)} MB")
print(f"null values: {df.isna().sum().sum()}")

train_csv memory usage: 1893.4 MB
df memory usage: 60.27 MB
null values: 0


In [6]:
to_drop = []
for i in df.columns.tolist():
  if i not in data_columns:
    to_drop.append(i)

df_train = df.drop(to_drop, axis=1)
df_train.head(3)

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,f_37,f_38,f_39,f_40,f_41,f_42,f_43,f_44,f_45,f_46,f_47,f_48,f_49,f_50,f_51,f_52,f_53,f_54,f_55,f_56,f_57,f_58,f_59,f_60,f_61,f_62,f_63,f_64,f_65,f_66,f_67,f_68,f_69,f_70,f_71,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79,f_80,f_81,f_82,f_83,f_84,f_85,f_86,f_87,f_88,f_89,f_90,f_91,f_92,f_93,f_94,f_95,f_96,f_97,f_98,f_99,f_100,f_101,f_102,f_103,f_104,f_105,f_106,f_107,f_108,f_109,f_110,f_111,f_112,f_113,f_114,f_115,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,f_140,f_141,f_142,f_143,f_144,f_145,f_146,f_147,f_148,f_149,f_150,f_151,f_152,f_153,f_154,f_155,f_156,f_157,f_158,f_159,f_160,f_161,f_162,f_163,f_164,f_165,f_166,f_167,f_168,f_169,f_170,f_171,f_172,f_173,f_174,f_175,f_176,f_177,f_178,f_179,f_180,f_181,f_182,f_183,f_184,f_185,f_186,f_187,f_188,f_189,f_190,f_191,f_192,f_193,f_194,f_195,f_196,f_197,f_198,f_199,f_200,f_201,f_202,f_203,f_204,f_205,f_206,f_207,f_208,f_209,f_210,f_211,f_212,f_213,f_214,f_215,f_216,f_217,f_218,f_219,f_220,f_221,f_222,f_223,f_224,f_225,f_226,f_227,f_228,f_229,f_230,f_231,f_232,f_233,f_234,f_235,f_236,f_237,f_238,f_239,f_240,f_241,f_242,f_243,f_244,f_245,f_246,f_247,f_248,f_249,f_250,f_251,f_252,f_253,f_254,f_255,f_256,f_257,f_258,f_259,f_260,f_261,f_262,f_263,f_264,f_265,f_266,f_267,f_268,f_269,f_270,f_271,f_272,f_273,f_274,f_275,f_276,f_277,f_278,f_279,f_280,f_281,f_282,f_283,f_284,f_285,f_286,f_287,f_288,f_289,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,0.932617,0.113708,-0.402100,0.378418,-0.203979,-0.413574,0.965820,1.230469,0.114807,-2.013672,0.004936,0.284180,0.501953,-0.287842,-1.168945,-0.267334,-0.574219,-0.771973,1.012695,-1.230469,1.786133,-2.089844,0.325684,-0.877930,1.048828,0.131714,-0.349609,-1.813477,0.099243,-0.240967,1.604492,0.003637,-0.901855,0.221558,0.609863,-0.738770,2.097656,-0.914062,-0.293945,-0.037994,0.685547,0.000000,0.414795,-0.469482,-1.085938,-0.106445,0.059448,0.483154,1.189453,0.506836,0.754395,0.317627,-0.183228,5.164062,0.677246,-0.440674,0.631348,-1.845703,0.227783,0.841309,0.667480,-0.854004,-0.170410,-0.119690,-0.504883,0.662598,1.933594,-0.812500,0.072815,-0.778809,-0.553711,-0.032776,-0.619141,-1.424805,0.148315,0.257812,0.735352,0.563965,-0.272217,1.190430,-0.101379,1.333984,0.191284,0.630859,1.010742,0.161865,0.609375,-0.219482,-0.013069,1.505859,-1.319336,0.448242,-0.635254,-0.075378,1.033203,-0.664062,0.139893,0.186279,0.593262,-0.301514,0.666504,0.713379,1.020508,0.879883,-1.096680,-0.211060,-0.117493,-0.778809,0.065979,0.229736,0.435303,1.233398,-0.865723,0.062347,0.382324,-1.315430,0.786621,0.770508,-0.871094,0.583496,0.115601,0.209595,0.325439,0.469238,0.000000,-0.731445,-0.249512,-0.050568,-1.232422,-0.790527,0.493408,0.357910,-0.381592,0.176392,0.748047,-0.741699,0.089661,1.080078,1.137695,1.199219,0.030609,-0.356201,0.247803,1.395508,0.961914,0.078125,-1.163086,0.707520,0.256104,-0.026306,0.329102,0.196045,0.653320,0.069214,0.907715,-0.350586,-1.170898,0.659180,0.937500,0.482910,-0.094910,0.317383,0.126831,0.892090,-1.199219,-0.315430,0.704102,-0.017700,-0.513184,-0.362549,1.000000,2.265625,0.360107,0.054474,1.639648,-1.000000,0.876953,0.368164,0.991211,-1.224609,-1.208984,-0.879883,1.000000,0.035156,0.693359,0.307129,0.143799,0.728027,1.220703,-0.939941,-0.106934,0.000000,-0.569336,0.187866,-0.386475,-0.452393,0.269775,-0.565430,-0.262207,-1.473633,0.000000,0.428223,1.740234,0.941406,-0.729980,-0.533203,-0.983398,-0.043152,-0.225952,-0.008301,-0.447998,0.551270,-0.153076,-2.292969,0.186646,-0.443115,0.121216,0.787598,1.996094,-1.286133,-0.157227,1.019531,0.693359,0.797363,-0.192627,0.222778,0.921387,-0.171387,0.021652,1.153320,0.689941,3.251953,0.992188,-0.779785,-0.546387,-0.379150,1.896484,-0.732422,-0.113770,1.127930,-0.007927,0.122314,0.444580,0.485840,-2.302734

In [7]:
df_investment_id = df["investment_id"]
integer_lookup_layer = tf.keras.layers.IntegerLookup()
integer_lookup_layer.adapt(pd.DataFrame({"investment_id": list(df_investment_id.unique())}))

df_target = pd.DataFrame(df["target"])
df_target.head(3)

,target
0,-0.300875
1,-0.231040
2,0.568807


In [23]:
def create_model_1():
  activation_func = tf.keras.activations.relu
  investment_id_input = tf.keras.layers.Input(shape=(1,))
  x_input = tf.keras.layers.Input(shape=(len(data_columns),))
  x = tf.keras.layers.Dense(128, activation=activation_func)(x_input)
  x = tf.keras.layers.Dense(128, activation=activation_func)(x)

  investment_id_x = integer_lookup_layer(investment_id_input)
  investment_id_x = tf.keras.layers.Embedding(len(df_investment_id.unique()) + 1, 32, input_length=1)(investment_id_x) # shape (None, 1, 32)
  investment_id_x = tf.keras.layers.Reshape((-1, ))(investment_id_x) # shape (None, 32)
  investment_id_x = tf.keras.layers.Dense(128, activation=activation_func)(investment_id_x)
  investment_id_x = tf.keras.layers.Dense(128, activation=activation_func)(investment_id_x)

  concat = tf.keras.layers.Concatenate(axis=1)([investment_id_x, x])
  concat = tf.keras.layers.Dense(128, activation=activation_func)(concat)
  concat = tf.keras.layers.Dense(32, activation=activation_func)(concat)
  output = tf.keras.layers.Dense(1)(concat)

  model = Model(inputs=[investment_id_input, x_input], outputs=[output])
  model.compile(
    optimizer=tf.keras.optimizers.Adam(), 
    loss=tf.keras.losses.MeanSquaredError(), 
    metrics=tf.keras.metrics.RootMeanSquaredError()
  )
  return model

In [24]:
def create_ds(X: pd.DataFrame, investment_id: pd.DataFrame,  y: pd.DataFrame):
  ds = tf.data.Dataset.from_tensor_slices(((investment_id, X), y["target"]))
  ds = ds.shuffle(128)
  ds = ds.batch(32)
  ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  return ds

epochs = 5
random_state = 1
n_splits = 3
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
split = kfold.split(df_train[data_columns], df_investment_id)

for i, (train_index, test_index) in enumerate(split):
  X_train, X_test = df_train.iloc[train_index], df_train.iloc[test_index]
  y_train, y_test = df_target.iloc[train_index], df_target.iloc[test_index]
  investment_id_train, investment_id_test = df_investment_id.iloc[train_index], df_investment_id.iloc[test_index]
  ds_train = create_ds(X_train, investment_id_train, y_train)
  ds_test = create_ds(X_test, investment_id_test, y_test)
  model = create_model_1()
  model.fit(ds_train, epochs=epochs, validation_data=ds_test, callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
      filepath = f"model_split_{i}",
      save_best_only=True,
    )
  ])

  del ds_train, ds_test, model
  gc.collect()

Epoch 1/5
2084/2084 [==============================] - 17s 8ms/step - loss: 0.8393 - root_mean_squared_error: 0.9161 - val_loss: 0.9624 - val_root_mean_squared_error: 0.9810
Epoch 2/5
2084/2084 [==============================] - 16s 8ms/step - loss: 0.8223 - root_mean_squared_error: 0.9068 - val_loss: 0.9551 - val_root_mean_squared_error: 0.9773
Epoch 3/5
2084/2084 [==============================] - 13s 6ms/step - loss: 0.7680 - root_mean_squared_error: 0.8763 - val_loss: 1.0502 - val_root_mean_squared_error: 1.0248
Epoch 4/5
2084/2084 [==============================] - 14s 7ms/step - loss: 0.7318 - root_mean_squared_error: 0.8555 - val_loss: 1.1526 - val_root_mean_squared_error: 1.0736
Epoch 5/5
2084/2084 [==============================] - 13s 6ms/step - loss: 0.7031 - root_mean_squared_error: 0.8385 - val_loss: 1.1872 - val_root_mean_squared_error: 1.0896
Epoch 1/5
2084/2084 [==============================] - 16s 7ms/step - loss: 0.8338 - root_mean_squared_error: 0.9131 - val_loss: 1

In [33]:
def predict(models, ds):
  predictions = []
  for model in models:
    predictions.append(model.predict(ds))
  return np.mean(predictions, axis=0)

def create_predict_ds(X: pd.DataFrame, investment_id: pd.DataFrame):
  ds = tf.data.Dataset.from_tensor_slices(((investment_id, X)))
  ds = ds.batch(32)
  ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  return ds

models = []

for i in range(n_splits):
  models.append(tf.keras.models.load_model(f"model_split_{i}"))

df_test = None # insert df here
ds_test = create_predict_ds = create_predict_ds(df_test)
predict(models, ds_test)